# Import Dependencies

In [1]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt

import re
from nltk.corpus import stopwords
from textblob import TextBlob

%matplotlib inline

# Read into Python (Kaggle Dataset)

In [2]:
#load dataset
df = pd.read_csv('/Users/madhukumar/Desktop/Portfolio/github/glassdoor-sentiment/data/employee_reviews.csv')
df.head(5)

,Unnamed: 0,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link
0,1,google,none,"Dec 11, 2018",Current Employee - Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,none,5.0,4.0,5.0,5.0,4.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...
1,2,google,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,4.0,2.0,3.0,3.0,5.0,3.0,2094,https://www.glassdoor.com/Reviews/Google-Revie...
2,3,google,"New York, NY","May 10, 2014",Current Employee - Software Engineer III,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5.0,5.0,4.0,5.0,5.0,4.0,949,https://www.glassdoor.com/Reviews/Google-Revie...
3,4,google,"Mountain View, CA","Feb 8, 2015",Current Employee - Anonymous Employee,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,5.0,2.0,5.0,5.0,4.0,5.0,498,https://www.glassdoor.com/Reviews/Google-Revie...
4,5,google,"Los Angeles, CA","Jul 19, 2018",Former Employee - Software Engineer,"Unique, one of a kind dream job",Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",Promote managers into management for their man...,5.0,5.0,5.0,5.0,5.0,5.0,49,https://www.glassdoor.com/Reviews/Google-Revie...


In [3]:
df.summary.fillna(' ', inplace=True)
df.pros.fillna(' ', inplace=True)
df.cons.fillna(' ', inplace=True)
df['advice-to-mgmt'].fillna(' ', inplace=True)

In [4]:
df['full_review'] = df['summary'] + '. ' + df['pros'] + '. ' + df ['cons'] + '. ' + df['advice-to-mgmt'] + '.'

# Data Inspection

In [5]:
df.full_review.replace(' ', np.nan, inplace=True)

In [6]:
df.shape

(67529, 18)

In [7]:
df.rename(columns={'overall-ratings':'rating', 'job-title':'title'}, inplace=True)

In [8]:
#removing unwanted columns
df.drop(['Unnamed: 0','link', 'summary', 'pros','cons', 'advice-to-mgmt', 
        'work-balance-stars','culture-values-stars', 'carrer-opportunities-stars',
        'comp-benefit-stars', 'senior-mangemnet-stars','helpful-count', 'company'], axis=1, inplace=True)

In [9]:
df.head(3)

,location,dates,title,rating,full_review
0,none,"Dec 11, 2018",Current Employee - Anonymous Employee,5.0,Best Company to work for. People are smart and...
1,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,4.0,"Moving at the speed of light, burn out is inev..."
2,"New York, NY","May 10, 2014",Current Employee - Software Engineer III,5.0,Great balance between big-company security and...


# Text Preprocessing

In [10]:
def preprocess(summary):
    summary = summary.str.replace("(<br/>)", "")
    summary = summary.str.replace('(<a).*(>).*(</a>)', '')
    summary = summary.str.replace('(&amp)', '')
    summary = summary.str.replace('(&gt)', '')
    summary = summary.str.replace('(&lt)', '')
    summary = summary.str.replace('(\xa0)', ' ')
    #removing punctuation
    summary = summary.str.replace('[^\w\s]','')
    summary = summary.str.replace('\d+', '')

    return summary

df['clean_review'] = preprocess(df['full_review'])

In [11]:
df.head(4)

,location,dates,title,rating,full_review,clean_review
0,none,"Dec 11, 2018",Current Employee - Anonymous Employee,5.0,Best Company to work for. People are smart and...,Best Company to work for People are smart and ...
1,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,4.0,"Moving at the speed of light, burn out is inev...",Moving at the speed of light burn out is inevi...
2,"New York, NY","May 10, 2014",Current Employee - Software Engineer III,5.0,Great balance between big-company security and...,Great balance between bigcompany security and ...
3,"Mountain View, CA","Feb 8, 2015",Current Employee - Anonymous Employee,5.0,The best place I've worked and also the most d...,The best place Ive worked and also the most de...


In [12]:
%%time
#lowercasing
df['clean_review'] = df['clean_review'].str.lower()  

CPU times: user 73.8 ms, sys: 14.6 ms, total: 88.4 ms
Wall time: 98.1 ms


In [13]:
df.head(3)

,location,dates,title,rating,full_review,clean_review
0,none,"Dec 11, 2018",Current Employee - Anonymous Employee,5.0,Best Company to work for. People are smart and...,best company to work for people are smart and ...
1,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,4.0,"Moving at the speed of light, burn out is inev...",moving at the speed of light burn out is inevi...
2,"New York, NY","May 10, 2014",Current Employee - Software Engineer III,5.0,Great balance between big-company security and...,great balance between bigcompany security and ...


# Language Detection

In [14]:
import langid

In [15]:
# get the language id for each text
ids_langid = df['full_review'].apply(langid.classify)

# get just the language label
langs = ids_langid.apply(lambda tuple: tuple[0])

# how many unique language labels were applied
print("Number of tagged languages (estimated):")
print(len(langs.unique()))

# percent of the total dataset in English
print("Percent of data in English (estimated):")
print((sum(langs=="en")/len(langs))*100)

Number of tagged languages (estimated):
27
Percent of data in English (estimated):
99.12037791171201


In [16]:
# convert our list of languages to a dataframe
langs_df = pd.DataFrame(langs)

# count the number of times we see each language
langs_count = langs_df.full_review.value_counts()

In [17]:
print("Languages with more than 400 tweets in our dataset:")
print(langs_count[langs_count > 400])

print("")

print("Percent of our dataset in these languages:")
print((sum(langs_count[langs_count > 400])/len(langs)) * 100)

Languages with more than 400 tweets in our dataset:
en    66935
Name: full_review, dtype: int64

Percent of our dataset in these languages:
99.12037791171201


In [18]:
# get just the language label
df['language'] = ids_langid.apply(lambda tuple: tuple[0])

In [19]:
df.head(3)

,location,dates,title,rating,full_review,clean_review,language
0,none,"Dec 11, 2018",Current Employee - Anonymous Employee,5.0,Best Company to work for. People are smart and...,best company to work for people are smart and ...,en
1,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,4.0,"Moving at the speed of light, burn out is inev...",moving at the speed of light burn out is inevi...,en
2,"New York, NY","May 10, 2014",Current Employee - Software Engineer III,5.0,Great balance between big-company security and...,great balance between bigcompany security and ...,en


In [20]:
df.language.value_counts()

en    66935
fr      235
de      207
nl       65
af       21
es       11
it       10
id        9
pt        4
ca        4
no        3
ro        3
lt        3
da        2
sq        2
tr        2
eu        2
mg        2
cy        1
et        1
br        1
tl        1
la        1
fo        1
pl        1
an        1
vo        1
Name: language, dtype: int64

In [21]:
df = df[df['language'] == 'en']

In [22]:
df.shape

(66935, 7)

# Cleaning Dates

In [23]:
#removing punctuation
df['dates'] = df['dates'].str.replace('[^\w\s]','')
#converting to date format
df['datetime'] = pd.to_datetime(df['dates'], format=' %b %d %Y', errors='coerce')
#Extracting Year
df['year'] = df['datetime'].dt.year
#Extracting the weekday name of the date
df['day'] = df['datetime'].dt.day_name()
#Extracting month
df['month'] = df['datetime'].dt.month

In [24]:
df.head()

,location,dates,title,rating,full_review,clean_review,language,datetime,year,day,month
0,none,Dec 11 2018,Current Employee - Anonymous Employee,5.0,Best Company to work for. People are smart and...,best company to work for people are smart and ...,en,2018-12-11,2018.0,Tuesday,12.0
1,"Mountain View, CA",Jun 21 2013,Former Employee - Program Manager,4.0,"Moving at the speed of light, burn out is inev...",moving at the speed of light burn out is inevi...,en,2013-06-21,2013.0,Friday,6.0
2,"New York, NY",May 10 2014,Current Employee - Software Engineer III,5.0,Great balance between big-company security and...,great balance between bigcompany security and ...,en,2014-05-10,2014.0,Saturday,5.0
3,"Mountain View, CA",Feb 8 2015,Current Employee - Anonymous Employee,5.0,The best place I've worked and also the most d...,the best place ive worked and also the most de...,en,2015-02-08,2015.0,Sunday,2.0
4,"Los Angeles, CA",Jul 19 2018,Former Employee - Software Engineer,5.0,"Unique, one of a kind dream job. Google is a w...",unique one of a kind dream job google is a wor...,en,2018-07-19,2018.0,Thursday,7.0


# Cleaning Locations

In [25]:
#removing punctuation characters
df['location'] = df['location'].str.replace('[^\w\s]','')
df['clean_location'] = df['location'].str.extract(r'\b(\w+)$', expand=True)
#Replacing US states with US country label
df['clean_location'] = df['clean_location'].str.replace('[A-Z]+[A-Z]', 'United States of America', regex=True)

df.rename(columns={'clean_location':'country'}, inplace=True)

In [26]:
df['country'] = df['country'].str.replace("Republic", "Czech Republic")
df['country'] = df['country'].str.replace("Taiwan", "Taiwan, Province of China")
df['country'] = df['country'].str.replace("Korea", "Korea, Democratic People's Republic of")
df['country'] = df['country'].str.replace("Arabia", "Saudi Arabia")
df['country'] = df['country'].str.replace("Faso", "Burkina Faso")
df['country'] = df['country'].str.replace("Burkina Burkina Faso", "Burkina Faso")
df['country'] = df['country'].str.replace("Saudi Saudi Arabia", "Saudi Arabia")
df['country'] = df['country'].str.replace("Russia", "Russian Federation")
df['country'] = df['country'].str.replace("Venezuela", "Venezuela, Bolivarian Republic of")
df['country'] = df['country'].str.replace("Vietnam", "Viet Nam")
df['country'] = df['country'].str.replace("Africa", "Central African Republic")
df['country'] = df['country'].str.replace("Kong", "Hong Kong")
df['country'] = df['country'].str.replace("Lanka", "Sri Lanka")
df['country'] = df['country'].str.replace("Emirates", "United Arab Emirates")
df['country'] = df['country'].str.replace("Iran", "Iran, Islamic Republic of")
df['country'] = df['country'].str.replace("Rica", "Costa Rica")
df['country'] = df['country'].str.replace("Islands", "Cayman Islands")
df['country'] = df['country'].str.replace("Tanzania", "Tanzania, United Republic of")
df['country'] = df['country'].str.replace("Timor", "Timor-Leste")
df['country'] = df['country'].str.replace("dIvoire", "Côte d'Ivoire")
df['country'] = df['country'].str.replace("Libya", "Libyan Arab Jamahiriya")
df['country'] = df['country'].str.replace("Salvador", "El Salvador")
df['country'] = df['country'].str.replace("Herzegovina", "Bosnia and Herzegovina")
df['country'] = df['country'].str.replace("Zealand", "New Zealand")

# Feature Extraction - Employee Job Titles

In [27]:
Current = df['title'].str.contains('Current', na='False') 
Former = df['title'].str.contains('Former', na='False') 

print (Current.value_counts())
print (Former.value_counts())

True     42201
False    24734
Name: title, dtype: int64
False    42199
True     24736
Name: title, dtype: int64


In [28]:
Anonymous = df['title'].str.contains('Anonymous', na='False') 
Anonymous.value_counts()

False    40180
True     26755
Name: title, dtype: int64

In [29]:
df['title'] = df['title'].str.replace('[^\w\s]','')

In [30]:
df['anonymity'] = df['title'].str.extract(pat='(Anonymous)',expand=False)
# map existing values to a different set of values
df['anonymity'] = df.anonymity.map({'Anonymous':1})
df['anonymity'].fillna(2, inplace=True)

In [31]:
df['employee_type'] = df['title'].str.extract(pat='(Current|Former)',expand=False)
# map existing values to a different set of values
df['employee_type'] = df.employee_type.map({'Current':1, 'Former': 2})

# Export to CSV

Saving the clean data back to disk for future use

In [32]:
#removing unwanted columns
df.drop(['language', 'title', 'dates', 'datetime', 'location'], axis=1, inplace=True)

In [33]:
df.to_csv("/Users/madhukumar/Desktop/Portfolio/github/glassdoor-sentiment/data/employee_reviews_clean.csv")
df.head()

,rating,full_review,clean_review,year,day,month,country,anonymity,employee_type
0,5.0,Best Company to work for. People are smart and...,best company to work for people are smart and ...,2018.0,Tuesday,12.0,none,1.0,1
1,4.0,"Moving at the speed of light, burn out is inev...",moving at the speed of light burn out is inevi...,2013.0,Friday,6.0,United States of America,2.0,2
2,5.0,Great balance between big-company security and...,great balance between bigcompany security and ...,2014.0,Saturday,5.0,United States of America,2.0,1
3,5.0,The best place I've worked and also the most d...,the best place ive worked and also the most de...,2015.0,Sunday,2.0,United States of America,1.0,1
4,5.0,"Unique, one of a kind dream job. Google is a w...",unique one of a kind dream job google is a wor...,2018.0,Thursday,7.0,United States of America,2.0,2
